In [1]:
import os
import csv
from matplotlib import pyplot as plt

samples=[]
with open('./data/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [2]:
from sklearn.model_selection import train_test_split
train_samples,validation_samples=train_test_split(samples,test_size=0.2)

In [ ]:
import cv2
import numpy as np
import sklearn

def generator(samples,batch_size=32):
    num_samples=len(samples)
    correction=0.2
    while 1:
#         shuffle(samples)
        for offset in range(0,num_samples,batch_size):
            batch_samples=samples[offset:offset+batch_size]
            
            images=[]
            angles=[]
            for batch_sample in batch_samples:
                
                name=batch_sample[0].split('/')[-1]
                left_name=batch_sample[1].split('/')[-1]
                right_name=batch_sample[2].split('/')[-1]
                print(left_name)
                left_image=cv2.imread(left_name)
                right_image=cv2.imread(right_name)
                
                center_image=cv2.imread(name)
                center_angle=float(batch_sample[3])
                
                left_angle=center_angle+correction
                right_angle=center_angle-correction
                
                image_flipped = np.fliplr(center_image)
                measurement_flipped = -center_angle
                
                images.append(center_image)
                angles.append(center_angle)
                
                images.append(image_flipped)
                angles.append(measurement_flipped)
                
                
                images.append(left_image)
                angles.append(left_angle)
                
                images.append(right_image)
                angles.append(right_angle)
            
            X_train=np.array(images)
            y_train=np.array(angles)
            yield sklearn.utils.shuffle(X_train,y_train)
            

In [ ]:
train_generator=generator(train_samples,batch_size=32)
validation_generator=generator(validation_samples,batch_size=32)

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,Cropping2D,BatchNormalization,Activation,Dropout,MaxPooling2D,Lambda
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

c:\programdata\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
ACTIVATION='relu'
model = Sequential()
model.add(Lambda(lambda x: (x / 127.5) - 1., input_shape = (160, 320, 3)))
model.add(Cropping2D(cropping=((74,24), (0,0))))
model.add(Conv2D(24,5,5,subsample=(2,2),activation=ACTIVATION))
model.add(Conv2D(36,5,5,subsample=(2,2),activation=ACTIVATION))
model.add(Conv2D(48,5,5,subsample=(2,2),activation=ACTIVATION))
model.add(Conv2D(64,3,3,activation=ACTIVATION))
model.add(Conv2D(64,3,3,activation=ACTIVATION))
model.add(Flatten())
model.add(Dropout(.5))
model.add(Dense(1164))
model.add(Activation(ACTIVATION))
model.add(Dense(100))
model.add(Activation(ACTIVATION))
model.add(Dense(50))
model.add(Activation(ACTIVATION))
model.add(Dense(10))
model.add(Activation(ACTIVATION))
model.add(Dense(1))
model.summary()
# model=Sequential()
# model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(160,320,3)))
# model.add(Cropping2D(cropping=((74,24),(0,0))))
# model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
# model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
# model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
# model.add(Conv2D(64, 3, 3, activation='elu'))
# model.add(Conv2D(64, 3, 3, activation='elu'))
# model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(100, activation='elu'))
# model.add(Dense(50, activation='elu'))
# model.add(Dense(10, activation='elu'))
# model.add(Dense(1))
# model.summary()

c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`


Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 62, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 35, 64)         27712     
_________________________________________________________________
con

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=2)
checkpointer = ModelCheckpoint(filepath='model-{val_loss:.5f}.h5', verbose=1, save_best_only=True)
model.compile(loss='mse', optimizer='adam')
fit_loss = model.fit_generator(train_generator, steps_per_epoch=len(train_samples),
                    validation_data=validation_generator,
            validation_steps=len(validation_samples), nb_epoch=5,callbacks=[early_stopping,checkpointer],verbose=1)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=6389, validation_data=<generator..., validation_steps=1598, callbacks=[<keras.ca..., verbose=1, epochs=5)`
  


Epoch 1/5
6389/6389 [==============================] - 1592s 249ms/step - loss: 0.0179 - val_loss: 0.0232
Epoch 2/5
6389/6389 [==============================] - 1159s 181ms/step - loss: 0.0086 - val_loss: 0.0219
Epoch 3/5
 858/6389 [===>..........................] - ETA: 13:10 - loss: 0.0077

In [ ]:
plt.plot(fit_loss.history['loss'])
plt.plot(fit_loss.history['val_loss'])
plt.title('Mean Squared Error Loss')
plt.ylabel('mean squared error')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
model.save('model.h5')